In [19]:
import numpy as np
import matplotlib.pyplot as plt

def asset_paths_gbm(S0, r, sigma, T, NSamples, NRepl, K=None, plot=True, antithetic=False):
    dt = T / NSamples
    paths = np.zeros((NSamples + 1, NRepl))
    paths[0] = S0
    
    if antithetic:
        paths_anti = np.zeros((NSamples + 1, NRepl))
        paths_anti[0] = S0
    
    for t in range(1, NSamples + 1):
        Z = np.random.normal(0, 1, NRepl)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
        
        if antithetic:
            paths_anti[t] = paths_anti[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt - sigma * np.sqrt(dt) * Z)
    
    if plot:
        plt.figure(figsize=(10, 6))
        plt.plot(paths, alpha=0.1, color='blue')
        if antithetic:
            plt.plot(paths_anti, alpha=0.1, color='green')
        plt.xlabel("Time steps")
        plt.ylabel("Asset price")
        plt.title("Geometric Brownian Motion (GBM) Paths")
        plt.grid()
        
        if K is not None:
            plt.axhline(y=K, color='red', linestyle='--', label=f"Strike Price (K={K})")
            plt.legend()
        
        plt.show()
    
    return (paths, paths_anti) if antithetic else paths

def mc_arithmetic_asian(S0, K, r, sigma, T, NSamples, NRepl, antithetic=False):
    paths = asset_paths_gbm(S0, r, sigma, T, NSamples, NRepl, K=K, plot=False, antithetic=antithetic)
    payoffs = np.zeros(NRepl)
    
    if antithetic:
        payoffs_anti = np.zeros(NRepl)
        paths, paths_anti = paths
    
    for i in range(NRepl):
        avg_price = np.mean(paths[1:, i])
        payoffs[i] = max(0, avg_price - K)
        
        if antithetic:
            avg_price_anti = np.mean(paths_anti[1:, i])
            payoffs_anti[i] = max(0, avg_price_anti - K)
    
    discount_factor = np.exp(-r * T)
    
    if antithetic:
        payoffs_combined = (payoffs + payoffs_anti) / 2
        mean_price = discount_factor * np.mean(payoffs_combined)
        std_error = np.std(payoffs_combined) / np.sqrt(NRepl)
    else:
        mean_price = discount_factor * np.mean(payoffs)
        std_error = np.std(payoffs) / np.sqrt(NRepl)
    
    z_score = 1.96  # 95% confidence interval
    ci_lower = mean_price - z_score * std_error
    ci_upper = mean_price + z_score * std_error
    
    return mean_price, (ci_lower, ci_upper)

# Example parameters
S0, K, r, sigma, T, NSamples, NRepl = 100, 100, 0.05, 0.2, 1, 50, 10000

# Standard MC
price_mc, ci_mc = mc_arithmetic_asian(S0, K, r, sigma, T, NSamples, NRepl)
print(f"Standard MC - Option price: {price_mc:.4f}, Confidence interval: [{ci_mc[0]:.4f}, {ci_mc[1]:.4f}]")

# Antithetic Variates MC
price_av, ci_av = mc_arithmetic_asian(S0, K, r, sigma, T, NSamples, NRepl, antithetic=True)
print(f"Antithetic Variates MC - Option price: {price_av:.4f}, Confidence interval: [{ci_av[0]:.4f}, {ci_av[1]:.4f}]")


Standard MC - Option price: 5.8084, Confidence interval: [5.6443, 5.9724]
Antithetic Variates MC - Option price: 5.8426, Confidence interval: [5.7609, 5.9242]
